In [2]:
import torch 
import torch.nn as nn 
import matplotlib.pyplot as plt 
import numpy as np 
%matplotlib inline 

In [224]:
def findpadding(inputsize,outputsize,kernelsize,stride):
    padding=((stride*(outputsize-1))-(inputsize-kernelsize))/2
    return np.ceil(padding)
findpadding(224,112,7,2)

3.0

In [228]:
class residualblock(nn.Module):
    def __init__(self, inchannels, midchannels, outchannels, downsample):
        super(residualblock,self).__init__()
        if downsample:
            self.conv1 = nn.Conv2d(inchannels, midchannels, kernel_size=3,stride=2,padding=1,bias=False)
            self.down = nn.Sequential(
                nn.Conv2d(inchannels, midchannels, kernel_size=3,stride=2,padding=1),
                nn.BatchNorm2d(outchannels)
            )
        else:
            self.conv1 = nn.Conv2d(inchannels,midchannels, kernel_size=3,stride=1,padding=1,bias=False)
            self.down = nn.Identity()
        self.seq = nn.Sequential(
            nn.BatchNorm2d(midchannels),
            nn.ReLU(),
            nn.Conv2d(midchannels,outchannels,kernel_size=3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(outchannels)
        )
        self.relu = nn.ReLU()
    def forward(self,x):
        identity = x 
        identity = self.down(identity)
        x = self.seq(self.conv1(x)) 
        x+=identity
        return self.relu(x)
class ResNet18(nn.Module):
    def __init__(self,inchannels):
        super(ResNet18,self).__init__()
        self.seq1 = nn.Sequential(
            nn.Conv2d(inchannels,64, kernel_size=7,stride=2,padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        )
        self.l1 = nn.Sequential(
            residualblock(64,64,64,False),
            residualblock(64,64,64,False)
        )
        self.l2 = nn.Sequential(
            residualblock(64,128,128,True),
            residualblock(128,128,128,False)
        )
        self.l3 = nn.Sequential(
            residualblock(128,256,256,True),
            residualblock(256,256,256,False)
        )
        self.l4 = nn.Sequential(
            residualblock(256,512,512,True),
            residualblock(512,512,512,False)
        )
        self.l5 = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(512,1000)                   
        )
    def forward(self,x):
        x = self.seq1(x)
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = self.l5(x)
        return x
resnet18 = ResNet18(3)
print(resnet)

ResNet(
  (seq1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (l1): Sequential(
    (0): residualblock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (down): Identity()
      (seq): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU()
    )
    (1): residualblock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (down): Identity()
      (seq): Sequential(
        (0): Batc